In [1]:
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time

df = pd.read_excel('Hot Stuff.xlsx')

In [2]:
def retiraFeaturing(artistaString):
    
    if " Featuring" in artistaString:   # Se a música tiver um featuring, deve-se retirá-lo
            indexFeat = artistaString.find(" Featuring")   # pega índice do char space antes de Featuring
            artistaString = artistaString[:indexFeat]   # retorna apenas o primeiro artista
        
    return artistaString   # caso não caia no if, simplesmente retorna o artista assim como veio

In [3]:
#esta celula calcula o quao popular foi uma musica no mes especificado (Maio de 1990) atribuindo um sistema de pontos baseado em que posicao ela ficou no top 100 em cada semana do mês
#vai contar o quao popular foi a musica no ano atribuindo pontos a quantas semanas ela ficou entre as mais tocadas
#testando se o char especificado esta na entrada atual
#pode ser usado para identificar anos diferentes

tempo = time.time() #Armazena o tempo de inicio do processamento
topArtistas = {}
for ano in range(1960,2015):
    contador = {}
    #Salva os registros do ano atual
    for i in range(len(df)):
        
        if (str(ano) + '-') in df['WeekID'][i]: #filtra todas as entradas por ano E mes nesse caso (Maio de 1990)
            if(df['SongID'][i] in contador): #testa se a musica ja esta no dicionario (ja foi computada)
                contador[df['SongID'][i]]['Pontos'] += (101 - df['Week Position'][i])
            else: #cai aqui se ainda nao foi computada
                contador[df['SongID'][i]] = {}
                contador[df['SongID'][i]]['Artista'] = df['Performer'][i]
                contador[df['SongID'][i]]['Titulo'] = df['Song'][i]
                contador[df['SongID'][i]]['Ano'] = ano
                contador[df['SongID'][i]]['Pontos'] = (101 - df['Week Position'][i])
                #A medida 'Peak' nao eh a posicao exata que ela atingiu mas os "pontos maximos" ganhos em uma semana (de 1 a 100)
                contador[df['SongID'][i]]['Peak'] = (101 - df['Peak Position'][i]) #salva qual foi a maior posicao alcançada pela musica (usada para criterios de desempate)
                contador[df['SongID'][i]]['Semanas'] = df['Weeks on Chart'][i] #Terceiro criterio de desempate
        
        
    #Faz o sort do ano atual
    items = sorted(contador.items(), key = lambda tup: (tup[1]["Pontos"], tup[1]["Peak"], tup[1]["Semanas"]), reverse=True)
    
    #armazena as primeiras 200 entradas em um dicionário mais organizado
    top = [dict() for x in range(200)] #cria uma lista de 200 dicionarios
    for i in range(len(top)):
    
        for y in items[i][1]:
            top[i][y] = items[i][1][y]
        
        # Adiciona no dicionário de todos os artistas (ou soma os pontos) para cianção do arquivo topArtistas.bin
        artistaIterado = retiraFeaturing(str(top[i]['Artista']))   # retira o featuring caso tenha
        if artistaIterado in topArtistas:   # testa se o artista já está no dicionário
            topArtistas[artistaIterado] += top[i]['Pontos']   # se sim só soma nos seus pontos
        else:   # caso não esteja, adicona
            topArtistas[artistaIterado] = top[i]['Pontos']
     
    
    #Concatena o dicionario no arquivo teste.bin
    filename = 'teste.bin'
    topBytes = []
    

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            topBytes = pickle.load(openfile)


    newData = top #Aqui vem a nova data que deve ser concatenada
        
    topBytes = topBytes + newData #concatena o que ja tinha no arquivo binario com a nova data
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(topBytes, openfile) 
        
        
    #Cria o arquivo de indice para consultas    
    filename = 'indices.bin'
    fileBytes = []

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            fileBytes = pickle.load(openfile)


    newData = {'Ano': ano, 'Min': len(topBytes) - 200, 'Max': len(topBytes) - 1} #Aqui vem a nova data que deve ser concatenada
        
    fileBytes.append(newData)#concatena o que ja tinha no arquivo binario com a nova data
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(fileBytes, openfile)
    
    #Ordena o dicionário de artistas
    artistasSorted = sorted(topArtistas.items(), key=lambda kv: kv[1], reverse=True)
    
    #Salva ranking geral de artistas
    with (open("topArtistas.bin", 'wb+')) as openfile:
        pickle.dump(artistasSorted, openfile)
    
    
t = time.time() - tempo #Salva em t o tempo que o processo levou
print('Tempo levado para gerar os arquivos: ', t, 'segundos')

Tempo levado para gerar os arquivos:  308.3478789329529 segundos


In [4]:
#Faz a leitura do arquivo de registros
with (open('teste.bin', 'rb')) as openfile:
    leitura = pickle.load(openfile)
    
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

In [5]:
for i in indices:
    print(i)

{'Ano': 1960, 'Min': 0, 'Max': 199}
{'Ano': 1961, 'Min': 200, 'Max': 399}
{'Ano': 1962, 'Min': 400, 'Max': 599}
{'Ano': 1963, 'Min': 600, 'Max': 799}
{'Ano': 1964, 'Min': 800, 'Max': 999}
{'Ano': 1965, 'Min': 1000, 'Max': 1199}
{'Ano': 1966, 'Min': 1200, 'Max': 1399}
{'Ano': 1967, 'Min': 1400, 'Max': 1599}
{'Ano': 1968, 'Min': 1600, 'Max': 1799}
{'Ano': 1969, 'Min': 1800, 'Max': 1999}
{'Ano': 1970, 'Min': 2000, 'Max': 2199}
{'Ano': 1971, 'Min': 2200, 'Max': 2399}
{'Ano': 1972, 'Min': 2400, 'Max': 2599}
{'Ano': 1973, 'Min': 2600, 'Max': 2799}
{'Ano': 1974, 'Min': 2800, 'Max': 2999}
{'Ano': 1975, 'Min': 3000, 'Max': 3199}
{'Ano': 1976, 'Min': 3200, 'Max': 3399}
{'Ano': 1977, 'Min': 3400, 'Max': 3599}
{'Ano': 1978, 'Min': 3600, 'Max': 3799}
{'Ano': 1979, 'Min': 3800, 'Max': 3999}
{'Ano': 1980, 'Min': 4000, 'Max': 4199}
{'Ano': 1981, 'Min': 4200, 'Max': 4399}
{'Ano': 1982, 'Min': 4400, 'Max': 4599}
{'Ano': 1983, 'Min': 4600, 'Max': 4799}
{'Ano': 1984, 'Min': 4800, 'Max': 4999}
{'Ano': 1985

In [6]:
print(len(leitura))

11000


### Teste da relação
consultando o indice inicial de um dado ano no arquivo indice e imprimindo os 5 mais populares daquele ano

In [7]:
start = 0

for x in indices:
    if x['Ano'] == 2014:
        start = x['Min']
 
for i in range(5):
    print(leitura[start + i])

{'Artista': 'John Legend', 'Titulo': 'All Of Me', 'Ano': 2014, 'Pontos': 4069, 'Peak': 100, 'Semanas': 59}
{'Artista': 'Pharrell Williams', 'Titulo': 'Happy', 'Ano': 2014, 'Pontos': 3898, 'Peak': 100, 'Semanas': 47}
{'Artista': 'Katy Perry Featuring Juicy J', 'Titulo': 'Dark Horse', 'Ano': 2014, 'Pontos': 3816, 'Peak': 100, 'Semanas': 57}
{'Artista': 'OneRepublic', 'Titulo': 'Counting Stars', 'Ano': 2014, 'Pontos': 3373, 'Peak': 99, 'Semanas': 68}
{'Artista': 'Iggy Azalea Featuring Charli XCX', 'Titulo': 'Fancy', 'Ano': 2014, 'Pontos': 2993, 'Peak': 100, 'Semanas': 39}


In [8]:
start = 0

for x in indices:
    if x['Ano'] == 2014:
        start = x['Min']
        
#Retorna as posicoes de 100 a 104 
desvio = 100

for i in range(5):
    print(leitura[start + i + desvio])

{'Artista': 'Ty Dolla $ign Featuring B.o.B', 'Titulo': 'Paranoid', 'Ano': 2014, 'Pontos': 933, 'Peak': 72, 'Semanas': 20}
{'Artista': 'Fall Out Boy', 'Titulo': 'Centuries', 'Ano': 2014, 'Pontos': 924, 'Peak': 91, 'Semanas': 34}
{'Artista': 'Miley Cyrus', 'Titulo': 'Adore You', 'Ano': 2014, 'Pontos': 920, 'Peak': 80, 'Semanas': 18}
{'Artista': 'Nicki Minaj', 'Titulo': 'Pills N Potions', 'Ano': 2014, 'Pontos': 916, 'Peak': 77, 'Semanas': 18}
{'Artista': 'Miranda Lambert Duet With Carrie Underwood', 'Titulo': "Somethin' Bad", 'Ano': 2014, 'Pontos': 898, 'Peak': 82, 'Semanas': 20}


In [9]:
artistasSorted

[('Madonna', 53607),
 ('Elton John', 47983),
 ('Rihanna', 47334),
 ('Mariah Carey', 46615),
 ('Usher', 43886),
 ('Katy Perry', 37169),
 ('P!nk', 35094),
 ('Michael Jackson', 34889),
 ('Stevie Wonder', 34693),
 ('Taylor Swift', 34497),
 ('The Beatles', 34372),
 ('Whitney Houston', 34084),
 ('Rod Stewart', 33097),
 ('Chris Brown', 32323),
 ('Maroon 5', 32269),
 ('Chicago', 31620),
 ('Beyonce', 30098),
 ('Kelly Clarkson', 30064),
 ('Daryl Hall John Oates', 29523),
 ('The Rolling Stones', 29277),
 ('R. Kelly', 29116),
 ('Billy Joel', 28704),
 ('Janet Jackson', 28575),
 ('The Black Eyed Peas', 28203),
 ('Bee Gees', 28009),
 ('Britney Spears', 26239),
 ('Lady Gaga', 26145),
 ('Drake', 25691),
 ('Aretha Franklin', 25548),
 ('Tim McGraw', 25340),
 ('Eminem', 24376),
 ('Nickelback', 24367),
 ('Kenny Chesney', 23899),
 ('Justin Timberlake', 23766),
 ('Neil Diamond', 23611),
 ('Alicia Keys', 23187),
 ('Pitbull', 23138),
 ('Mary J. Blige', 22902),
 ('Nelly', 22666),
 ('The Beach Boys', 22573),
 ('